In [4]:
import unicodecsv
from datetime import datetime as dt

def get_recs(filename):
    with open(filename, 'rb') as f:
        f_reader = unicodecsv.DictReader(f)
        f_list = list(f_reader)
        return f_list
    
enrollments = get_recs('enrollments.csv')    
daily_engagement = get_recs('daily_engagement.csv')
project_submissions = get_recs('project_submissions.csv')

# Takes a date as a string, and returns a Python datetime object. 
# If there is no date given, returns None
def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')

# Takes a string which is either an empty string or represents an integer,
# and returns an int or None.    
def parse_maybe_int(i):
    if i == '':
        return None
    else:
        return int(i)

# Clean up the data types in the enrollments table
for enrollment in enrollments:
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['days_to_cancel'] = parse_maybe_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled'] == 'True'
    enrollment['is_udacity'] = enrollment['is_udacity'] == 'True'
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    
# Clean up the data types in the engagement table
for engagement_record in daily_engagement:
    engagement_record['lessons_completed'] = int(float(engagement_record['lessons_completed']))
    engagement_record['num_courses_visited'] = int(float(engagement_record['num_courses_visited']))
    engagement_record['projects_completed'] = int(float(engagement_record['projects_completed']))
    engagement_record['total_minutes_visited'] = float(engagement_record['total_minutes_visited'])
    engagement_record['utc_date'] = parse_date(engagement_record['utc_date'])

# Clean up the data types in the submissions table
for submission in project_submissions:
    submission['completion_date'] = parse_date(submission['completion_date'])
    submission['creation_date'] = parse_date(submission['creation_date'])
    
print "Enrollments:\n", enrollments[0],"\n"
print "Daily engagements:\n", daily_engagement[0], "\n"
print "Project Submissions:\n", project_submissions[0], "\n" 
    

Enrollments:
{u'status': u'canceled', u'is_udacity': True, u'is_canceled': True, u'join_date': datetime.datetime(2014, 11, 10, 0, 0), u'account_key': u'448', u'cancel_date': datetime.datetime(2015, 1, 14, 0, 0), u'days_to_cancel': 65} 

Daily engagements:
{u'lessons_completed': 0, u'num_courses_visited': 1, u'total_minutes_visited': 11.6793745, u'projects_completed': 0, u'acct': u'0', u'utc_date': datetime.datetime(2015, 1, 9, 0, 0)} 

Project Submissions:
{u'lesson_key': u'3176718735', u'processing_state': u'EVALUATED', u'account_key': u'256', u'assigned_rating': u'UNGRADED', u'completion_date': datetime.datetime(2015, 1, 16, 0, 0), u'creation_date': datetime.datetime(2015, 1, 14, 0, 0)} 



In [21]:
#####################################
#                 2                 #
#####################################

## Find the total number of rows and the number of unique students (account keys)
## in each table.
def table_summary(table):
    num_rows = len(table)
    student_ids = []
    for rec in table:
        if 'account_key' in rec:
            id = rec['account_key']
        elif 'acct' in rec:
            id = rec['acct']
        if id not in student_ids:
            student_ids.append(id)
    return {'row_ct': num_rows, 'student_ct': len(student_ids)}

print table_summary(enrollments)
print table_summary(daily_engagement)
print table_summary(project_submissions)

{'student_ct': 1302, 'row_ct': 1640}
{'student_ct': 1237, 'row_ct': 136240}
{'student_ct': 743, 'row_ct': 3642}


## Use Sets instead
While the code above works, it's more efficient to use a set, rather than check the list each time to see if the value is already in it.

In [27]:
## Find the total number of rows and the number of unique students (account keys)
## in each table.
def table_summary(table):
    '''Using sets instead of checking list. Returns set of student ids and row count of table'''
    num_rows = len(table)
    student_ids = set()
    for rec in table:
        if 'account_key' in rec:
            id = rec['account_key']
        elif 'acct' in rec:
            id = rec['acct']
        student_ids.add(id)
    return {'row_ct': num_rows, 'students': student_ids}

# NOTE THAT WE"LL WANT THESE LATER TO COMPARE SETS
enrollment_summary = table_summary(enrollments)
engagement_summary = table_summary(daily_engagement)
project_summary = table_summary(project_submissions)

for summary in [enrollment_summary, engagement_summary, project_summary]:
    print "Row count: %d, Student count: %d" % (summary['row_ct'], len(summary['students']))

Row count: 1640, Student count: 1302
Row count: 136240, Student count: 1237
Row count: 3642, Student count: 743


In [28]:
#####################################
#                 3                 #
#####################################

## Rename the "acct" column in the daily_engagement table to "account_key".

def key_rename(table, names):
    '''Rename the "acct" column in the daily_engagement table to "account_key"'''
    for rec in table:
        old_name = names['old']
        new_name = names['new']
        if old_name in rec:
            rec[new_name] = rec.pop(old_name)

key_rename(daily_engagement, {'old':'acct', 'new':'account_key'})
daily_engagement[0]

{'account_key': u'0',
 u'lessons_completed': 0,
 u'num_courses_visited': 1,
 u'projects_completed': 0,
 u'total_minutes_visited': 11.6793745,
 u'utc_date': datetime.datetime(2015, 1, 9, 0, 0)}

In [38]:
#####################################
#                 4                 #
#####################################

## Find any one student enrollments where the student is missing from the daily engagement table.
## Output that enrollment.

enrolled_students = enrollment_summary['students']
engaged_students = engagement_summary['students']
missing_students = [x for x in enrolled_students if x not in engaged_students]

for enrollment in enrollments:
    if enrollment['account_key'] == missing_students[0]:
        print enrollment



{u'status': u'canceled', u'is_udacity': False, u'is_canceled': True, u'join_date': datetime.datetime(2015, 4, 4, 0, 0), u'account_key': u'1145', u'cancel_date': datetime.datetime(2015, 4, 4, 0, 0), u'days_to_cancel': 0}


In [43]:
#####################################
#                 5                 #
#####################################

## Find the number of surprising data points (enrollments missing from
## the engagement table) that remain, if any. We're asking for the number of rows in the enrollments table 
## where the student stayed enrolled at least one day and there is no entry for that account key 
## in the daily engagement table
for enrollment in enrollments:
    if enrollment['account_key'] in missing_students:
        if enrollment['days_to_cancel'] > 0 or not enrollment['is_canceled']:
            print enrollment
        
        

{u'status': u'canceled', u'is_udacity': True, u'is_canceled': True, u'join_date': datetime.datetime(2015, 1, 10, 0, 0), u'account_key': u'1304', u'cancel_date': datetime.datetime(2015, 3, 10, 0, 0), u'days_to_cancel': 59}
{u'status': u'canceled', u'is_udacity': True, u'is_canceled': True, u'join_date': datetime.datetime(2015, 3, 10, 0, 0), u'account_key': u'1304', u'cancel_date': datetime.datetime(2015, 6, 17, 0, 0), u'days_to_cancel': 99}
{u'status': u'current', u'is_udacity': True, u'is_canceled': False, u'join_date': datetime.datetime(2015, 2, 25, 0, 0), u'account_key': u'1101', u'cancel_date': None, u'days_to_cancel': None}


In [54]:
# Create a set of the account keys for all Udacity test accounts
udacity_test_accounts = set()
for enrollment in enrollments:
    if enrollment['is_udacity']:
        udacity_test_accounts.add(enrollment['account_key'])
len(udacity_test_accounts)

# Given some data with an account_key field, removes any records corresponding to Udacity test accounts
def remove_udacity_accounts(data):
    non_udacity_data = []
    for data_point in data:
        if data_point['account_key'] not in udacity_test_accounts:
            non_udacity_data.append(data_point)
    return non_udacity_data

# Remove Udacity test accounts from all three tables
non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_engagement = remove_udacity_accounts(daily_engagement)
non_udacity_submissions = remove_udacity_accounts(project_submissions)

print len(non_udacity_enrollments)
print len(non_udacity_engagement)
print len(non_udacity_submissions)

print non_udacity_engagement[0]

1622
135656
3634
{u'lessons_completed': 0, u'num_courses_visited': 1, u'total_minutes_visited': 11.6793745, u'projects_completed': 0, 'account_key': u'0', u'utc_date': datetime.datetime(2015, 1, 9, 0, 0)}


In [52]:
## Create a dictionary named paid_students containing all students who either
## haven't canceled yet or who remained enrolled for more than 7 days. The keys
## should be account keys, and the values should be the date the student enrolled.
def find_paid_students():
    paid_students = {}
    for student in non_udacity_enrollments:
        if not student['is_canceled'] or student['days_to_cancel'] > 7:
            account_key = student['account_key'] 
            start_date = student['join_date']
            if account_key not in paid_students or start_date > paid_students[account_key]:
                paid_students[account_key] = start_date
    return paid_students

paid_students = find_paid_students()
len(paid_students)
print "{0:%s}" % (paid_students['0'])

{0:2015-01-09 00:00:00}


In [62]:
# Takes a student's join date and the date of a specific engagement record,
# and returns True if that engagement record happened within one week
# of the student joining.
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    return time_delta.days < 7 and time_delta.days >= 0

## Create a list of rows from the engagement table including only rows where
## the student is one of the paid students you just found, and the date is within
## one week of the student's join date.
def find_first_week_engagement(paid_students):
    engagement_list = []
    for student in paid_students:
        join_date = paid_students[student]
        for engagement in non_udacity_engagement:
            if student == engagement['account_key']:
                if within_one_week(join_date, engagement['utc_date']):
                    engagement_list.append(engagement)
    return engagement_list
            
paid_engagement_in_first_week = find_first_week_engagement(paid_students)
len(paid_engagement_in_first_week)

6919